<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Processing</a></span></li></ul></div>

In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs
import json, re, os
from tqdm import tqdm
from loguru import logger
from datetime import datetime
import pandas as pd

In [2]:
# feed_config.py and import
RSS_FEED_URLS = [
    'https://timesofindia.indiatimes.com/rssfeedstopstories.cms?x=1',
    'https://feeds.feedburner.com/NDTV-LatestNews',
    'https://www.indiatoday.in/rss/1206578',
    'https://indianexpress.com/feed/',
    'https://www.thehindu.com/news/national/?service=rss',
    'https://www.news18.com/rss/india.xml',
    'https://www.business-standard.com/rss/latest.rss',
    'https://www.dnaindia.com/feeds/india.xml',
    'https://www.deccanchronicle.com/rss_feed/',
    'https://www.india.com/feed/',
    'https://www.oneindia.com/rss/news-india-fb.xml',
    'https://news.abplive.com/home/feed',
    'https://theprint.in/feed/',
    'https://www.indiatvnews.com/rssnews/topstory.xml',
    'https://www.tribuneindia.com/rss/feed?catId=42',
    'https://www.thehindubusinessline.com/?service=rss',
#     'https://prod-qt-images.s3.amazonaws.com/production/freepressjournal/feed.xml', # no results
    'https://www.siasat.com/feed/',
    'https://feeds.feedburner.com/ScrollinArticles.rss',
    'https://telanganatoday.com/feed'
]

In [3]:
len(RSS_FEED_URLS)

19

In [4]:
session = HTMLSession()
response = session.get(RSS_FEED_URLS[0])
response.status_code

200

In [5]:
soup = bs(response.content, features='xml')

In [6]:
items  = soup.find_all('item')

In [7]:
len(items)

33

In [8]:
items[0]

<item><title>'Tip of the iceberg': 9,00,000 may have died due to Covid in 5 weeks in China</title><description>Using a report from the National School of Development at Peking University that found 64% of the population was infected by mid-January, a report estimated 900,000 people would have died in the previous five weeks based on a conservative 0.1% case fatality rate. That means the official hospital death count is less than 7% of the total mortality seen during the outbreak. </description><link>https://timesofindia.indiatimes.com/world/china/coronavirus-china-covid-death-zero-covid/articleshow/97023577.cms</link><guid>https://timesofindia.indiatimes.com/world/china/coronavirus-china-covid-death-zero-covid/articleshow/97023577.cms</guid><pubDate>Mon, 16 Jan 2023 13:06:49 +0530</pubDate><dc:creator>Bloomberg</dc:creator><enclosure length="2221629" type="image/jpeg" url="https://static.toiimg.com/photo/msid-97023635,imgsize-2221629.cms"/></item>

In [9]:
title = items[0].find('title').text
title

"'Tip of the iceberg': 9,00,000 may have died due to Covid in 5 weeks in China"

In [10]:
description = items[0].find('description').text
description

'Using a report from the National School of Development at Peking University that found 64% of the population was infected by mid-January, a report estimated 900,000 people would have died in the previous five weeks based on a conservative 0.1% case fatality rate. That means the official hospital death count is less than 7% of the total mortality seen during the outbreak. '

In [11]:
pub_date = items[0].find('pubDate').text
pub_date

'Mon, 16 Jan 2023 13:06:49 +0530'

In [12]:
all_records = []

articles = []

In [13]:
# main.py
def fetch_data(url):
    rss_feed_data = dict()
    try:
        session = HTMLSession()
        response = session.get(url)
        logger.info(f"Connected to: {url}")
    except Exception as e:
        logger.error(f"Unable to connect to: {url}")
    try:
        soup = bs(response.content, features='xml')
        items = soup.find_all('item')
        logger.info("All items fetched.")
    except Exception as e:
        logger.error("Unable to fetch items.")
    
    titles = [item.find('title').text for item in items]
    descriptions = [item.find('description').text for item in items]
    pub_dates = [item.find('pubDate').text for item in items]
    
    rss_feed_data['titles'] = titles
    rss_feed_data['descriptions'] = descriptions
    rss_feed_data['pub_dates'] = pub_dates
    
    return rss_feed_data

In [14]:
todays_feed = pd.concat([pd.DataFrame.from_dict(fetch_data(url)) for url in RSS_FEED_URLS], ignore_index=True)
print("\n", todays_feed.shape)
todays_feed.head()

2023-01-16 23:07:17.782 | INFO     | __main__:fetch_data:7 - Connected to: https://timesofindia.indiatimes.com/rssfeedstopstories.cms?x=1
2023-01-16 23:07:17.790 | INFO     | __main__:fetch_data:13 - All items fetched.
2023-01-16 23:07:19.013 | INFO     | __main__:fetch_data:7 - Connected to: https://feeds.feedburner.com/NDTV-LatestNews
2023-01-16 23:07:19.019 | INFO     | __main__:fetch_data:13 - All items fetched.
2023-01-16 23:07:19.291 | INFO     | __main__:fetch_data:7 - Connected to: https://www.indiatoday.in/rss/1206578
2023-01-16 23:07:19.309 | INFO     | __main__:fetch_data:13 - All items fetched.
2023-01-16 23:07:19.477 | INFO     | __main__:fetch_data:7 - Connected to: https://indianexpress.com/feed/
2023-01-16 23:07:19.578 | INFO     | __main__:fetch_data:13 - All items fetched.
2023-01-16 23:07:19.877 | INFO     | __main__:fetch_data:7 - Connected to: https://www.thehindu.com/news/national/?service=rss
2023-01-16 23:07:19.909 | INFO     | __main__:fetch_data:13 - All items


 (1700, 3)


,titles,descriptions,pub_dates
0,"'Tip of the iceberg': 9,00,000 may have died d...",Using a report from the National School of Dev...,"Mon, 16 Jan 2023 13:06:49 +0530"
1,"Not only Joshimath, these places are also at r...",,"Mon, 16 Jan 2023 09:33:46 +0530"
2,Kerala makes masks mandatory in public places,The Kerala government has made wearing masks c...,"Mon, 16 Jan 2023 21:11:40 +0530"
3,'2023 very important': BJP holds key meet afte...,Prime Minister Narendra Modi held a mega roads...,"Mon, 16 Jan 2023 16:56:17 +0530"
4,"My surgery was a success, ready for the challe...","India wicketkeeper-batter Rishabh Pant, who su...","Mon, 16 Jan 2023 19:27:05 +0530"


In [15]:
date = datetime.today().strftime("%Y%m%d")

folder_name = "./raw_data/"
filename = f"{date}_snapshot.json"

if not os.path.exists(folder_name):
    os.mkdir(folder_name)

with open(f"{folder_name}{filename}", 'w', encoding='utf-8') as file:
    todays_feed.to_json(file, orient='records', lines=True)

In [16]:
pd.read_json('./raw_data/20230116_snapshot.json', lines=True)

,titles,descriptions,pub_dates
0,"'Tip of the iceberg': 9,00,000 may have died d...",Using a report from the National School of Dev...,"Mon, 16 Jan 2023 13:06:49 +0530"
1,"Not only Joshimath, these places are also at r...",,"Mon, 16 Jan 2023 09:33:46 +0530"
2,Kerala makes masks mandatory in public places,The Kerala government has made wearing masks c...,"Mon, 16 Jan 2023 21:11:40 +0530"
3,'2023 very important': BJP holds key meet afte...,Prime Minister Narendra Modi held a mega roads...,"Mon, 16 Jan 2023 16:56:17 +0530"
4,"My surgery was a success, ready for the challe...","India wicketkeeper-batter Rishabh Pant, who su...","Mon, 16 Jan 2023 19:27:05 +0530"
...,...,...,...
1695,Golden Globes 2023: ‘The Fabelmans’ bags Best ...,"<figure><img width=""768"" height=""450"" src=""htt...","Wed, 11 Jan 2023 12:15:08 +0530"
1696,"Sensex declines 309.7 points to 59,805.78 in e...","<figure><img width=""768"" height=""450"" src=""htt...","Wed, 11 Jan 2023 12:00:29 +0530"
1697,‘It was a beautiful torture’: Ram Charan looks...,"<figure><img width=""768"" height=""450"" src=""htt...","Wed, 11 Jan 2023 11:45:08 +0530"
1698,"“There will be no third World War, it is not a...","<figure><img width=""768"" height=""450"" src=""htt...","Wed, 11 Jan 2023 11:30:17 +0530"
